In [1]:
using CSV
using DataFrames
using XLSX
using Statistics

In [34]:
P1_1 = CSV.read("../processed-data/Phylum/1 _ 1 .csv", DataFrame)
P1_2 = CSV.read("../processed-data/Phylum/4 _ 4 .csv", DataFrame)
#P1_3 = CSV.read("../processed-data/Phylum/1 _ 3 .csv", DataFrame)
#P1_4 = CSV.read("../processed-data/Phylum/1 _ 4 .csv", DataFrame);

In [35]:
responese = CSV.read("../processed-data/no_tuber_scabpit.csv", DataFrame)
P1_1 = innerjoin(P1_1, responese, on = :Column1)
P1_2 = innerjoin(P1_2, responese, on = :Column1)
#P1_3 = innerjoin(P1_3, responese, on = :Column1)
#P1_4 = innerjoin(P1_4, responese, on = :Column1);

In [36]:
P1_1 = P1_1[:, Not(44:45)]
P1_1 = P1_1[:, 2:44];

In [37]:
P1_2 = P1_2[:, Not(44:45)]
P1_2 = P1_2[:, 2:44];

In [6]:
#P1_3 = P1_3[:, Not(44:45)]
#P1_3 = P1_3[:, 2:44];

In [7]:
#P1_4 = P1_4[:, Not(44:45)]
#P1_4 = P1_4[:, 2:44];

In [38]:
a = Matrix(P1_1)
b = Matrix(P1_2)
#c = Matrix(P1_3)
#d = Matrix(P1_4);

In [39]:
CSV.write("../processed-data/P1_1SP.csv",  Tables.table(a), writeheader=false)
CSV.write("../processed-data/P4_4SP.csv",  Tables.table(b), writeheader=false)
#CSV.write("../processed-data/P1_3.csv",  Tables.table(c), writeheader=false)
#CSV.write("../processed-data/P1_4.csv",  Tables.table(d), writeheader=false)

"../processed-data/P4_4SP.csv"

In [40]:
a = CSV.read("../processed-data/P1_1SP.csv", DataFrame)
b = CSV.read("../processed-data/P4_4SP.csv", DataFrame)
percentage = [(i, count(==(i), a[1:171,43])) for i in unique(a[1:171,43])]

2-element Vector{Tuple{Int64, Int64}}:
 (1, 63)
 (0, 108)

In [41]:
m = Matrix(a)
n = Matrix(b);

In [42]:
CSV.write("../processed-data/P1_1SP.csv",  Tables.table(m), writeheader=false)
CSV.write("../processed-data/P4_4SP.csv",  Tables.table(n), writeheader=false)

"../processed-data/P4_4SP.csv"